<a href="https://colab.research.google.com/github/emely3h/Geospatial_ML/blob/main/scripts/make_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script for making predictions

### 0. Prepare Colab, Define Constants

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/MachineLearning/
#! git clone https://github.com/emely3h/Geospatial_ML.git
%cd Geospatial_ML
! ls
! git pull

/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning
fatal: destination path 'Geospatial_ML' already exists and is not an empty directory.
/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning/Geospatial_ML
data_exploration  experiments	     models	   pyproject.toml    scripts
docs		  image_processing   poetry.lock   README.md	     sripts
evaluation	  metrics_bug.ipynb  prepare_data  requirements.txt
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), 9.86 KiB | 9.00 KiB/s, done.
From https://github.com/emely3h/Geospatial_ML
   af24be7..f189b3a  main       -> origin/main
You are not currently on a branch.
Please specify which branch you want to merge with.
See git-pull(1) for details.

    git pull <remote> <branch>



In [3]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    concatenate,
    Conv2DTranspose,
    Dropout,
    UpSampling2D
)
from keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pickle
from keras.utils import Sequence
from datetime import datetime
from models.unet_model import unet_2d
from evaluation.evaluation_metrics import EvaluationMetrics
from data_exploration.mask_stats import Mask_Stats
from tensorflow.keras.models import load_model
from models.helpers import save_metrics, predictions_for_models
from evaluation.helpers import plot_metrics, load_metrics_into_df, calc_save_metrics_pred
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [4]:
total_tiles = 11121
train_tiles = 6672
test_val_tiles = 2224
data_path = "../data_colab/256_256"
experiment = "experiment_8"
batch_size = 32
tile_size = 256
step_size = 256

### 1. Create Data Generators

In [5]:
train_split_x = np.memmap(os.path.join(data_path, "train_split_x.npy"), mode="r", shape=(train_tiles, 256, 256, 5), dtype=np.uint8)
train_split_y = np.memmap(os.path.join(data_path, "train_split_y.npy"), mode="r", shape=(train_tiles, 256, 256), dtype=np.uint8)
val_split_x = np.memmap(os.path.join(data_path, "val_split_x.npy"), mode="r", shape=(test_val_tiles, 256, 256, 5), dtype=np.uint8)
val_split_y = np.memmap(os.path.join(data_path, "val_split_y.npy"), mode="r", shape=(test_val_tiles, 256, 256), dtype=np.uint8)
test_split_x = np.memmap(os.path.join(data_path, "test_split_x.npy"), mode="r", shape=(test_val_tiles, 256, 256, 5), dtype=np.uint8)
test_split_y = np.memmap(os.path.join(data_path, "test_split_y.npy"), mode="r", shape=(test_val_tiles, 256, 256), dtype=np.uint8)

train_stats = Mask_Stats(train_split_y)
train_stats.print_stats()
print()
val_stats = Mask_Stats(val_split_y)
val_stats.print_stats()
print()
test_stats = Mask_Stats(test_split_y)
test_stats.print_stats()

Shape: (6672, 256, 256)
Land pixels: 195058814  44.610 %
Valid pixels: 138904480  31.767 %
Invalid pixels: 103292898  23.623 %
Sum: 6672

Shape: (2224, 256, 256)
Land pixels: 65320265  44.816 %
Valid pixels: 46246663  31.730 %
Invalid pixels: 34185136  23.454 %
Sum: 2224

Shape: (2224, 256, 256)
Land pixels: 64786699  44.450 %
Valid pixels: 46892391  32.173 %
Invalid pixels: 34072974  23.377 %
Sum: 2224


In [6]:
class DataGenerator(Sequence):
    def __init__(self, mmap_x, mmap_y, batch_size):
        self.x_input = mmap_x
        self.y_mask = mmap_y
        self.batch_size = batch_size
        self.num_samples = self.x_input.shape[0]

    # returns number of batches as int
    def __len__(self):
        return int(np.ceil(self.num_samples / float(self.batch_size)))

    # returns single batch
    def __getitem__(self, index):
        batch_indices = slice(index * self.batch_size, (index + 1) * self.batch_size)
        batch_inputs = self.x_input[batch_indices]
        batch_masks = self.y_mask[batch_indices]

        # normalization
        batch_inputs = batch_inputs/255
        # one-hot-encoding
        batch_masks = np.array([tf.one_hot(item, depth=3).numpy() for item in batch_masks])

        # normalization + one hot encoding
        return batch_inputs, batch_masks

    def getitem_as_img(self, index):
        batch_indices = slice(index * self.batch_size, (index + 1) * self.batch_size)
        batch_inputs = self.x_input[batch_indices]
        batch_masks = self.y_mask[batch_indices]
        # normalization + one hot encoding
        return batch_inputs, batch_masks

In [7]:
# instanciate DataGenerators
batch_size = 32

train_generator = DataGenerator(train_split_x, train_split_y, batch_size)
val_generator = DataGenerator(val_split_x, val_split_y, batch_size)
test_generator = DataGenerator(test_split_x, test_split_y, batch_size)

print(train_generator.__len__())
print(val_generator.__len__())
print(test_generator.__len__())

209
70
70


In [8]:
train_batch = train_generator.__getitem__(9)
val_batch = val_generator.__getitem__(3)
test_batch = test_generator.__getitem__(4)

def print_batch_shapes(batch):
  print(type(batch))
  print(batch[0].shape)
  print(batch[1].shape)
  print()

# check batch shapes
print_batch_shapes(train_batch)
print_batch_shapes(val_batch)
print_batch_shapes(test_batch)

# check normalization
print('Check normalization')
print(train_batch[1].max())
print(train_batch[1].min())

print(val_batch[1].max())
print(val_batch[1].min())

print(test_batch[1].max())
print(test_batch[1].min())

print()
# check one-hot-encoding
print('check one hot encoding')
print(train_batch[0].max())
print(train_batch[0].min())

print(val_batch[0].max())
print(val_batch[0].min())

print(test_batch[0].max())
print(test_batch[0].min())

<class 'tuple'>
(32, 256, 256, 5)
(32, 256, 256, 3)

<class 'tuple'>
(32, 256, 256, 5)
(32, 256, 256, 3)

<class 'tuple'>
(32, 256, 256, 5)
(32, 256, 256, 3)

Check normalization
1.0
0.0
1.0
0.0
1.0
0.0

check one hot encoding
1.0
0.0
1.0
0.0
1.0
0.0


### 2. Make predictions
Use standard GPU, standard RAM

In [10]:
def get_filenames(experiment):
    files = os.listdir(f'../models/{experiment}/')
    return [f for f in files if f.startswith('model_')]
print(get_filenames(experiment))

18


In [13]:
predictions_for_models(train_generator, val_generator, test_generator, experiment, train_tiles, test_val_tiles, test_val_tiles, batch_size, (16,18))

start: 2023-06-14 12:54:24.430197
All found models: ['model_256_256_2_1.h5', 'model_256_256_3_1.h5', 'model_256_256_4_1.h5', 'model_256_256_5_1.h5', 'model_256_256_6_1.h5', 'model_256_256_7_1.h5', 'model_256_256_8_1.h5', 'model_256_256_3_2.h5', 'model_256_256_4_2.h5', 'model_256_256_5_2.h5', 'model_256_256_6_2.h5', 'model_256_256_7_2.h5', 'model_256_256_1_1.h5', 'model_256_256_2_3.h5', 'model_256_256_3_3.h5', 'model_256_256_4_3.h5', 'model_256_256_5_3_layers_64_128_256_512_1024.h5', 'model_256_256_3_4_layers_64_128_256.h5']
Make predictions with model model_256_256_5_3_layers_64_128_256_512_1024.h5
Start predictions with test data...
save pred in mmap: ../models/experiment_8/predictions/pred_test_1024.npy
1/1 [==============================] - 2s 2s/step
Start predictions with validation data...
save pred in mmap: ../models/experiment_8/predictions/pred_val_1024.npy
1/1 [==============================] - 0s 47ms/step
Start predictions with training data...

save pred in mmap: ../models